In [1]:
import cv2
from batch_face import (
    RetinaFace,
    drawLandmark_multiple,
    LandmarkPredictor,
    SixDRep
)
import os

In [2]:
def draw_landmarks(faces, landmarks, poses, img):
    head_pose = SixDRep(0)
    for face, landmark, pose in zip(faces, landmarks, poses[0]):
        img = drawLandmark_multiple(img, face[0], landmark)
        head_pose.plot_pose_cube(img, face[0], pose['pitch'], pose['yaw'], pose['roll'])
    return img

In [3]:
if __name__ == "__main__":
    output_dir = "./batch_img_results"
    os.makedirs(output_dir, exist_ok=True)
    predictor = LandmarkPredictor(0, "PFLD")
    detector = RetinaFace(0)
    head_pose = SixDRep(0)
    all_images = []
    names = os.listdir("batch_img_intake")

    for name in names:
        img = cv2.imread(os.path.join("batch_img_intake", name))
        all_images.append(img)

    all_faces = detector.pseudo_batch_detect(
        all_images, cv=True, threshold=0.9
    )  # batch input
    all_results = predictor(all_faces, all_images, from_fd=True)
    poses = head_pose(all_faces, all_images)

    assert len(all_results) == len(all_faces)

    for faces, landmarks, img, name in zip(all_faces, all_results, all_images, names):
        assert len(faces) == len(landmarks)
        img = draw_landmarks(faces,landmarks,poses,img)
        cv2.imwrite(os.path.join(output_dir, name), img)